In [1]:
import igl
import scipy as sp
import scipy.spatial as spatial
import numpy as np
import meshplot as mp
from scipy.spatial.transform import Rotation as R
from scipy.spatial.transform import Slerp

import os

os.chdir("/home/jovyan/")
root_folder = os.getcwd()

## Shape deformation using the Bi-Laplacian

The code below implements shape deformation by minimizing the bending energy using bi-harmonic displacement functions, that is, a displacement function whose Bi-Laplacian (Laplace of Laplace) is zero everywhere. You will notice that `igl` has a built-in function `harmonic_weights` to perform handle-based deformation using this approach. See also the [relevant chapter in the libigl tutorial](https://libigl.github.io/libigl-python-bindings/tutorials/#chapter-3-shape-deformation), where the code below is from.

## Assignment

Answer the following questions about the code below in a few sentences each:
- What do the red, purple, and green colored areas on the mesh indicate? How are these regions related to the system of equations shown in slide 13 from today's class?
- What is the meaning of the last parameter of `igl.harmonic_weights`? How is it related to the visualization on slide 12? Describe intuitively what type of results you get when you set the parameter to either 1 or 2.
- Describe the difference between whether the "deformation field" box is checked or not. In particular, how is this related to the visualization on slide 12?
- Explain one disadvantage of this approach to compute shape deformations.

Answers:
 - The red, purple and green areas are handles. In the code, red is handle 0 which does not move, purple is handle 1 which moves down, and green is the other handle which moves forward. These regions are analogous to the green region in slide 13 which forms the positional constraints in the right side of the equation, the $h_i$'s
 - The last parameter indicates the power of the harmonic operation, so 2 is biharmonic whereas 1 is harmonic. This relates to slide 12 since the middle set of images on that slide correspond with a parameter of 1 and the right set of images correspond with a parameter of 2. So, intuitively, 1 minimizes surface area (which becomes very clear when deformation field is turned off) and 2 minimizes bending.
 - The deformation field indicates whether we do the calculations with a deformation function or on the vertex positions directly. On slide 12, the bottom row corresponds to having the box checked and the top row corresponds to having the box unchecked.
 - This approach, since it's linear, does not always accurately represent natural behavior. For example, if the handle only translates, the solution will only translate. As described in the slides, this can cause the output to not rotate even though it naturally should.

In [3]:
v, f = igl.read_triangle_mesh(os.path.join(root_folder, "data", "decimated-max.obj"))
v[:,[0, 2]] = v[:,[2, 0]] # Swap X and Z axes
u = v.copy()

s = igl.read_dmat(os.path.join(root_folder, "data", "decimated-max-selection.dmat"))
b = np.array([[t[0] for t in [(i, s[i]) for i in range(0, v.shape[0])] if t[1] >= 0]]).T

## Boundary conditions directly on deformed positions
u_bc = np.zeros((b.shape[0], v.shape[1]))
v_bc = np.zeros((b.shape[0], v.shape[1]))

for bi in range(b.shape[0]):
    v_bc[bi] = v[b[bi]]

    if s[b[bi]] == 0: # Don't move handle 0
        u_bc[bi] = v[b[bi]]
    elif s[b[bi]] == 1: # Move handle 1 down
        u_bc[bi] = v[b[bi]] + np.array([[0, -50, 0]])
    else: # Move other handles forward
        u_bc[bi] = v[b[bi]] + np.array([[-25, 0, 0]])

p = mp.plot(v, f, s, shading={"wireframe": False, "colormap": "tab10"}, return_plot=True)

@mp.interact(deformation_field=True, step=(0.0, 2.0))
def update(deformation_field, step=0.0):
    # Determine boundary conditions
    u_bc_anim = v_bc + step * (u_bc - v_bc)

    if deformation_field:
        d_bc = u_bc_anim - v_bc
        d = igl.harmonic_weights(v, f, b, d_bc, 2)
        u = v + d
    else:
        u = igl.harmonic_weights(v, f, b, u_bc_anim, 2)
    p.update_object(vertices=u)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(82.297483…

interactive(children=(Checkbox(value=True, description='deformation_field'), FloatSlider(value=0.0, descriptio…

## Interpolating Rotations

The code below interpolates between a rotation around the y-axis by 0 and 180 degrees by linearly interpolating the two rotation matrices, which clearly leads to artifacts (move the slider to rotate the cow).

## Assignment

Change the code to interpolate the rotations using spherical linear interpolation (SLERP). Note that you can represent rotations as quaternions using [scipy.spatial.transform.Rotation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html). Scipy also provides an [implementation of SLERP](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Slerp.html). You need to solve this assignment without using the built-in function, but you may check correctness of your approach by comparing to it.

In [4]:
v, f = igl.read_triangle_mesh(os.path.join(root_folder, "data", "cow.off"))
steps = 100

# Using scipy's SLERP for comparison
rots = R.from_euler('y',[0,180],degrees=True)
slerp = Slerp([0,100], rots)
interp_rots = slerp(list(range(100)))

rot_0 = np.eye(3)
rot_1 = spatial.transform.Rotation.from_euler('y', 180, degrees=True)

# Can precalculate properties of the rotation quaternions (only do it for q_1 since q_0 is identity)
rotvec = rot_1.as_rotvec()
omega = sp.linalg.norm(rotvec) / 2
r = rotvec / (omega * 2)
v_rot = [v]
for i in range(steps):
    #t = i/(steps-1) # Bug in the original code I think (step 0 and step 1 were equal)
    t = (i+1)/(steps-1)
    rotquat_t = np.concatenate((r*np.sin(t*omega),[np.cos(t*omega)]))
    rot_t = R.from_quat(rotquat_t)
    rotmat_t = rot_t.as_matrix()
    #rotmat_t = interp_rots[i].as_matrix()
    v_t = v@rotmat_t
    v_rot.append(v_t)

p = mp.plot(v_rot[0], f, return_plot=True)

@mp.interact(t=(0, steps))
def mcf(t=0):
    p.update_object(vertices=v_rot[t])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

interactive(children=(IntSlider(value=0, description='t'), Output()), _dom_classes=('widget-interact',))